In [1]:
import requests
import json

file = open('API_KEY.txt', 'r')
API_KEY = file.read()

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(json.dumps(odds_json, indent=4))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 23
[
    {
        "id": "938a4536f9133fa3e2640b59c9849a98",
        "sport_key": "baseball_ncaa",
        "sport_title": "NCAA Baseball",
        "commence_time": "2024-04-30T21:30:00Z",
        "home_team": "Florida Gators",
        "away_team": "Florida Atlantic Owls",
        "bookmakers": [
            {
                "key": "mybookieag",
                "title": "MyBookie.ag",
                "last_update": "2024-04-30T23:58:00Z",
                "markets": [
                    {
                        "key": "h2h",
                        "last_update": "2024-04-30T23:58:00Z",
                        "outcomes": [
                            {
                                "name": "Florida Atlantic Owls",
                                "price": 3.9
                            },
                            {
                                "name": "Florida Gators",
                                "price": 1.22
                            }
              

In [2]:
# Event object == individual sporting event
class Event:
    def __init__(self, data):
        self.data = data
        self.sport_key = data['sport_key']
        self.id = data['id']

    def num_of_outcomes(self):  # Might need to change this unsure
        bookmakers = self.data['bookmakers']
        total = []
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']
            total.extend(odds)

        num_outcomes = len(total) # I think this is currently counting every possible outcome
        return num_outcomes

    def pull_odds(self): #
        master_list = []

        bookmakers = self.data['bookmakers']
        for bookmaker in bookmakers:
            odds = bookmaker['markets'][0]['outcomes']

            for outcome in odds:

                team_name = outcome['name']
                odds_price = outcome['price']
                bookie = bookmaker['title']

                info = {
                    "team name": team_name,
                    "odds" : odds_price,
                    "bookmaker" : bookie,
                }
                
                master_list.append(info)
        return master_list
    
    def calculate_best_odds(self, list): # | list = master_list 

        outcome_1_name = None
        outcome_2_name = None
        outcome_3_name = None

        outcome_1_best_odds = float("-inf")
        outcome_2_best_odds = float("-inf")
        outcome_3_best_odds = float("-inf")

        outcome_1_bookmaker = None
        outcome_2_bookmaker = None
        outcome_3_bookmaker = None

        for outcome in range(len(list)):
            
            if outcome_1_name is None or outcome_1_name == list[outcome]['team name']:
                outcome_1_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_1_best_odds:
                    outcome_1_best_odds = list[outcome]['odds']
                    outcome_1_bookmaker = list[outcome]['bookmaker']
            
            elif outcome_2_name is None or outcome_2_name == list[outcome]['team name']:
                outcome_2_name = list[outcome]['team name']
                if list[outcome]['odds'] > outcome_2_best_odds:
                    outcome_2_best_odds = list[outcome]['odds']
                    outcome_2_bookmaker = list[outcome]['bookmaker']
            
            if outcome_3_name is None and outcome_1_name != list[outcome]['team name'] and outcome_2_name != list[outcome]['team name']:
                outcome_3_name = list[outcome]['team name']

            if outcome_3_name == list[outcome]['team name']:
                if list[outcome]['odds'] > outcome_3_best_odds:
                    outcome_3_best_odds = list[outcome]['odds']
                    outcome_3_bookmaker = list[outcome]['bookmaker']

        return [[outcome_1_name, outcome_1_best_odds, outcome_1_bookmaker], [outcome_2_name, outcome_2_best_odds, outcome_2_bookmaker], [outcome_3_name, outcome_3_best_odds, outcome_3_bookmaker]]

        # Find best odds for each outcome of a matchup and return the [Team, Odds, Bookie] for each outcome

    # Calculate arbitrage | Boolean
    def calculate_arbitrage(self, best_odds_list):
        # If only 2 outcomes remove the 3rd item
        if best_odds_list[2][0] == None:
            best_odds_list.pop(2)
        else:
            return None # Will add 3 way arb

        odds_outcome_1 = best_odds_list[0][1]
        odds_outcome_2 = best_odds_list[1][1]

        arbitrage = ((1 / odds_outcome_1) * 100) + ((1 / odds_outcome_2) * 100)
        
        return arbitrage
    
    def arbitrage_present(self, arb):
        if arb == None:
            return None # Get rid of this after adding 3 way arb
        return arb < 100
    
    def american_to_decimal(self, american_odds):
        if american_odds > 0:
            return (american_odds / 100) + 1
        elif american_odds < 0:
            return (100 / american_odds) + 1
        else:
            return None # invalid odds
        
    def decimal_to_american(self, decimal_odds):
        if decimal_odds >= 2:
            return (decimal_odds - 1) * 100
        elif 1.01 <= decimal_odds < 2:
            return -100 / (decimal_odds - 1)
        else:
            return None # invalid odds
        
    # Calculate arbitrage bet return | [Multiplier]
    def arbitrage_bet_return(self, bet_amount, arbitrage_percent):
        if arbitrage_percent == None:
            return None
        return (bet_amount / (arbitrage_percent / 100)) - bet_amount

In [13]:
# Add class to append data to spreadsheet w/ pandas
for i in range(len(odds_json)):
    game = Event(odds_json[i])
    master_list = game.pull_odds()
    num_outcomes = game.num_of_outcomes()
    odds = game.calculate_best_odds(master_list)
    arb = game.calculate_arbitrage(odds)

    if game.arbitrage_present(arb):
        print("YAY")
        print(odds)
        print(game.decimal_to_american(round(odds[0][1], 0)))
        print(game.decimal_to_american(round(odds[1][1], 0)))
        print(round(game.arbitrage_bet_return(100, arb), 2)) # Gives money return for $100 total bets

YAY
[['Florida Atlantic Owls', 8.5, 'FanDuel'], ['Florida Gators', 1.22, 'MyBookie.ag']]
700.0
None
7 2
YAY
[['New York Knicks', 2.05, 'MyBookie.ag'], ['Philadelphia 76ers', 3.55, 'FanDuel']]
100.0
300.0
30 2
YAY
[['Boston Red Sox', 1.22, 'MyBookie.ag'], ['San Francisco Giants', 6.0, 'FanDuel']]
None
500.0
1 2
YAY
[['Chicago Cubs', 2.2, 'PointsBet (US)'], ['New York Mets', 1.91, 'BetMGM']]
100.0
100.0
2 2
YAY
[['Sam Houston St Bearkats', 4.9, 'FanDuel'], ['Texas Longhorns', 1.3, 'MyBookie.ag']]
400.0
None
3 2
YAY
[['Chicago White Sox', 2.1, 'MyBookie.ag'], ['Minnesota Twins', 1.97, 'Unibet']]
100.0
100.0
2 2
